# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of Word2Vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

C:\Users\Matthew Parker\.conda\envs\learn-env\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\Matthew Parker\.conda\envs\learn-env\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [3]:
raw_df = pd.read_json('News_Category_Dataset_v2.json', lines=True)

In [4]:
display(raw_df.head())
raw_df.info()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
authors              200853 non-null object
category             200853 non-null object
date                 200853 non-null datetime64[ns]
headline             200853 non-null object
link                 200853 non-null object
short_description    200853 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [9]:
raw_df['combined_text'] = raw_df.headline + ' ' + raw_df.short_description
data = raw_df.combined_text.map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [10]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [11]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [12]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55563350, 67352790)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, store this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [13]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [14]:
wv.most_similar('Texas')

[('Louisiana', 0.8014825582504272),
 ('Massachusetts', 0.7932997345924377),
 ('Michigan', 0.7894920706748962),
 ('Pennsylvania', 0.7888069152832031),
 ('Ohio', 0.7882728576660156),
 ('Illinois', 0.7793596982955933),
 ('Arkansas', 0.7781983613967896),
 ('Oregon', 0.7767912149429321),
 ('Maryland', 0.7759121060371399),
 ('Georgia', 0.774023175239563)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [15]:
wv.most_similar(negative='Texas')

[('Likened', 0.46052515506744385),
 ('Parent/Grandparent', 0.42248761653900146),
 ('much-vaunted', 0.414606511592865),
 ('Headstrong', 0.40347084403038025),
 ('End-of-School', 0.3949900269508362),
 ('maitre', 0.370331346988678),
 ('Re-Marry', 0.3652651309967041),
 ('Per-Mile', 0.36489832401275635),
 ('intercultural', 0.3645346164703369),
 ('Begets', 0.3607647120952606)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [17]:
wv['Texas']

array([-0.46860403, -0.7019416 ,  0.53557706,  0.998984  , -0.08391303,
        0.99245644,  0.6289039 , -2.6614883 ,  0.7638715 , -0.77866983,
       -1.6224071 ,  0.6196231 ,  0.773716  , -1.1606989 ,  1.0256166 ,
       -1.3459665 ,  0.30886546,  0.67840815,  0.70762867,  1.5365131 ,
        2.6055496 , -1.4842206 ,  1.4865668 , -0.94972754,  1.1021724 ,
       -0.5029818 ,  0.14968453, -2.4150023 , -1.1816133 , -0.9455985 ,
        1.4317636 ,  0.44618058, -0.1286432 , -1.9824865 ,  1.8456398 ,
       -0.5235245 ,  3.5798914 , -1.9013302 ,  0.8672037 , -1.1192216 ,
       -0.19145213, -0.27335724, -1.5800899 , -1.6811205 ,  0.46124706,
       -0.9614464 ,  1.0185095 ,  2.0495706 , -2.7477171 , -0.66268194,
        0.49065623, -0.28420255, -0.31852075,  2.0684373 , -0.2698643 ,
        1.2552391 ,  0.50952053, -0.07170533, -1.91078   , -0.01067701,
       -1.1244806 , -0.10625424,  0.7597061 ,  0.15530954, -0.2633869 ,
        1.2494195 , -0.67103136,  0.2787734 ,  1.0441055 , -1.64

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [18]:
wv.vectors

array([[ 1.7174317 ,  1.250918  ,  0.58109254, ..., -1.023696  ,
        -1.1845229 ,  0.02479874],
       [ 0.8146857 ,  0.57746613,  0.9840123 , ..., -1.6231788 ,
        -1.3378462 ,  0.92891556],
       [-2.1774132 ,  0.00312812,  0.75671136, ...,  0.2063793 ,
        -1.1970457 , -2.3547003 ],
       ...,
       [-0.06290412, -0.03480192,  0.02341756, ...,  0.03052284,
         0.06785346, -0.03150664],
       [-0.01214649, -0.00318389,  0.00524697, ...,  0.0599453 ,
        -0.06686332,  0.00671918],
       [-0.02989617, -0.05823547, -0.00784208, ..., -0.03769284,
         0.10623299, -0.00888838]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [28]:
wv.most_similar(positive=['king','woman'], negative=['man'])

[('crown', 0.617180347442627),
 ('villain', 0.5934091210365295),
 ('theme', 0.5872414708137512),
 ('queen', 0.5802618265151978),
 ('princess', 0.575808584690094),
 ('title', 0.5711683034896851),
 ('symbol', 0.5677425861358643),
 ('invention', 0.5659856796264648),
 ('enthusiast', 0.5646838545799255),
 ('revival', 0.5582786798477173)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!